In [1]:
from ogb.nodeproppred import PygNodePropPredDataset
import os
import os.path as osp
import torch_geometric
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import InMemoryDataset, download_url

from encoder import SentenceEncoder

In [2]:
# Loading the cora dataset
data_root = "../data"

In [3]:
class CoraPyGDataset(InMemoryDataset):
    def __init__(self, dataRoot="../data", custom_dataRoot="../custom_data", sentence_encoder=None, transform=None, pre_transform=None, pre_filter=None):
        self.data_root = dataRoot
        self.custom_data_root = custom_dataRoot
        self.sentence_encoder = sentence_encoder
        self.custom_data_dir = osp.join(self.custom_data_root, f"cora_{self.sentence_encoder.name}")

        if not osp.exists(self.custom_data_dir):
            os.makedirs(self.custom_data_dir)

        super().__init__(self.custom_data_dir, transform, pre_transform, pre_filter)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ["data.pt", "texts.pkl"]

    def text_to_embed(self, texts):
        if self.sentence_encoder is None:
            raise NotImplementedError("Sentence Encoder is not passed")
        if texts is None:
            return None
        else:
            return self.sentence_encoder.encode(texts)  # returns to self.device

    def encode_texts(self, texts):
        if isinstance(texts[0], str):
            return self.text_to_embed(texts)
        return [self.text_to_embed(t) for t in texts]

    def generate_custom_data(self):
        # Load the raw cora dataset
        data_path = osp.join(self.data_root, "cora", "cora.pt")
        raw_cora_data = torch.load(data_path)

        texts = raw_cora_data.raw_text
        label_names = raw_cora_data.label_names


        # Label and label description
        category_desc = pd.read_csv(osp.join(self.data_root, "cora", "categories.csv"), sep=",").values

        # Sort the label desc by the order of label_names
        ordered_desc = []
        for i, label in enumerate(label_names):
            true_ind = (label == category_desc[:, 0])
            ordered_desc.append((label, category_desc[true_ind, 1][0]))

        # Prompts for nodes/edges in original graph (can be changed accordingly)
        node_texts = ["Feature Node.\n Paper Title and abstract: " + t for t in texts]
        edge_text = ["Feature Edge.\n Connected papers are cited together by other papers."]

        # Node classification : Prompts for prompt node and label node (can be changed accordingly)
        prompt_node_text = ["Prompt Node.\n Node Classification on the paper's category"]
        label_texts = ["Prompt Node.\n Literature Category and Description: " + desc[0] + " + " + desc[1] for desc in ordered_desc]

        # Link prediction : Prompts for prompt node and edge labels (can be changed accordingly)
        prompt_node_edge_text = ["Prompt Node.\n Link Prediction on the papers that are cited together"]
        edge_label_text = ["Prompt Node.\n Two papers have co-citation",
                           "Prompt Node.\n Two papers do not have co-citation"]

        # Prompt for edge b/w prompt node and labels (can be changed accordingly)
        prompt_edge_text = ["Prompt Edge."]

        return raw_cora_data, [node_texts, label_texts, edge_text, prompt_node_edge_text, prompt_node_text, prompt_edge_text, edge_label_text]

    def process(self):
        # raw cora dataset is not in any library, so we process and load it manually in self.generate_custom_data()
        cora_data_list, texts = self.generate_custom_data()
        texts_embed = self.encode_texts(texts)

        torch.save(texts, self.processed_paths[1])

        cora_data_list.x_text_feat = texts_embed[0]
        cora_data_list.label_text_feat = texts_embed[1]
        cora_data_list.edge_text_feat = texts_embed[2]
        cora_data_list.prompt_text_edge_feat = texts_embed[3]
        cora_data_list.prompt_text_feat = texts_embed[4]
        cora_data_list.prompt_edge_feat = texts_embed[5]
        cora_data_list.edge_label_feat = texts_embed[6]

        # Pass the data_list as a list
        data, slices = self.collate([cora_data_list])

        torch.save((data, slices), self.processed_paths[0])
        print("Cora is processed. Saved.")

In [4]:
LMencoder = SentenceEncoder("ST", root="../lang_models", device=2)
custom_cora = CoraPyGDataset(dataRoot=data_root, sentence_encoder=LMencoder)

In [5]:
LMencoder2 = SentenceEncoder("roberta", root="../lang_models", device=2)
custom_cora2 = CoraPyGDataset(dataRoot=data_root, sentence_encoder=LMencoder2)

In [6]:
cora = custom_cora._data
cora

Data(raw_text=[2708], y=[2708], label_names=[7], edge_index=[2, 10858], train_masks=[10], val_masks=[10], test_masks=[10], x=[2708, 384], category_names=[2708], x_text_feat=[2708, 768], label_text_feat=[7, 768], edge_text_feat=[1, 768], prompt_text_edge_feat=[1, 768], prompt_text_feat=[1, 768], prompt_edge_feat=[1, 768], edge_label_feat=[2, 768])

In [7]:
cora.train_masks # it contains 10 different train masks (we will take avg of the accuracy)

[tensor([False,  True,  True,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False,  True,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False])]

In [8]:
custom_cora.y[custom_cora.train_masks[0]]

tensor([1, 2, 2, 0, 5, 4, 3, 1, 0, 4, 6, 2, 2, 6, 2, 1, 3, 0, 4, 5, 5, 5, 1, 0,
        1, 1, 3, 4, 4, 0, 3, 2, 2, 1, 2, 1, 0, 4, 6, 3, 6, 5, 6, 3, 2, 3, 0, 2,
        6, 3, 3, 1, 0, 5, 6, 1, 4, 0, 1, 1, 5, 6, 4, 2, 5, 3, 4, 2, 0, 5, 4, 2,
        1, 1, 5, 6, 5, 0, 2, 6, 4, 5, 1, 2, 0, 5, 2, 1, 3, 0, 4, 0, 0, 5, 4, 6,
        6, 6, 3, 3, 4, 3, 3, 3, 2, 5, 5, 6, 5, 6, 2, 3, 6, 3, 6, 5, 6, 1, 0, 6,
        4, 6, 3, 4, 4, 1, 0, 4, 1, 0, 2, 2, 5, 4, 3, 4, 0, 5, 0, 1])

In [9]:
custom_cora._data

Data(raw_text=[2708], y=[2708], label_names=[7], edge_index=[2, 10858], train_masks=[10], val_masks=[10], test_masks=[10], x=[2708, 384], category_names=[2708], x_text_feat=[2708, 768], label_text_feat=[7, 768], edge_text_feat=[1, 768], prompt_text_edge_feat=[1, 768], prompt_text_feat=[1, 768], prompt_edge_feat=[1, 768], edge_label_feat=[2, 768])

In [30]:
from torch_geometric.loader import DataLoader

dl = DataLoader(custom_cora, batch_size=12)

for batch in dl:
    print(batch)

DataBatch(raw_text=[1], y=[2708], label_names=[1], edge_index=[2, 10858], train_masks=[10], val_masks=[10], test_masks=[10], x=[2708, 384], category_names=[1], x_text_feat=[2708, 768], label_text_feat=[7, 768], edge_text_feat=[1, 768], prompt_text_edge_feat=[1, 768], prompt_text_feat=[1, 768], prompt_edge_feat=[1, 768], edge_label_feat=[2, 768], batch=[2708], ptr=[2])
